### Defs for Barplots

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


def plot_bars(data_dict, title, y_label):
    fig, ax = plt.subplots(figsize=(15, 7))

    names = list(data_dict.keys())
    values = [data_dict[name] for name in names]
    colors = sns.color_palette("Paired", len(names))

    for i, name in enumerate(names):
        name = name.split("_")[1] + " " + name.split("_")[2]
        name = name.replace("GPT", "GPT-2")
        bar = ax.bar(i, values[i], color=colors[i], label=name)
        ax.bar(name, values[i], color=colors[i], label=name)
        ax.text(
            bar[0].get_x() + bar[0].get_width() / 2,
            bar[0].get_height(),
            f"{values[i]:.2f}",
            ha="center",
            va="bottom",
        )
    ax.set_title(title)
    ax.set_ylabel(y_label)
    plt.xticks(rotation=30, ha="right", fontsize=10)
    plt.tight_layout()
    plt.savefig(f"./results/plots/{title}_barplot.svg")
    plt.show()
    plt.close(fig)


def plot_bars_avg(data_dict, title, y_label):
    fig, ax = plt.subplots(figsize=(15, 7))

    names = list(data_dict.keys())
    values = [data_dict[name] for name in names]
    colors = sns.color_palette("Paired", len(names))

    for i, name in enumerate(names):
        name = name.split("_")[0] + " " + name.split("_")[1]
        name = name.replace("GPT", "GPT-2")
        bar = ax.bar(i, values[i], color=colors[i], label=name)
        ax.bar(name, values[i], color=colors[i], label=name)
        ax.text(
            bar[0].get_x() + bar[0].get_width() / 2,
            bar[0].get_height(),
            f"{values[i]:.2f}",
            ha="center",
            va="bottom",
        )
    ax.set_title(title)
    ax.set_ylabel(y_label)
    plt.xticks(rotation=30, ha="right", fontsize=10)
    plt.tight_layout()
    plt.savefig(f"./results/plots/{title}_barplot.svg")
    plt.show()
    plt.close(fig)


def plot_bars_csv(data_dict, title, y_label):
    fig, ax = plt.subplots(figsize=(15, 7))

    names = list(data_dict.keys())
    values = [data_dict[name] for name in names]
    colors = sns.color_palette("Paired", len(names))

    for i, name in enumerate(names):
        name = name.split("_")[2] + " " + name.split("_")[4]
        name = name.replace("plus", "+")
        bar = ax.bar(i, values[i], color=colors[i], label=name)
        ax.bar(name, values[i], color=colors[i], label=name)
        ax.text(
            bar[0].get_x() + bar[0].get_width() / 2,
            bar[0].get_height(),
            f"{values[i]:.2f}",
            ha="center",
            va="bottom",
        )
    ax.set_title(title)
    ax.set_ylabel(y_label)
    plt.xticks(rotation=30, ha="right", fontsize=10)
    plt.tight_layout()
    plt.savefig(f"./results/plots/{title}_barplot.svg")
    plt.show()
    plt.close(fig)

# Validate a Model
Metrics: 
- Hard Position Accuracy
- Legal Piece Moves Accuracy
- Average Correct Plies

### Load Model

In [ ]:
from transformers import AutoModelForCausalLM

# modelname = "D:/LEON Safe/Leon-LLM-Models/xLANplus/Leon-Chess_350k_0001_4E_PLUS/Leon-Chess_350k_0001_4E_PLUS"  # Huggingface model name or local path


modelname = "Leon-LLM/Leon-Chess-71k-Plus"  # Huggingface model name or local path


model = AutoModelForCausalLM.from_pretrained(modelname)

### Generate validation prediction on Model

In [ ]:
from src.validation.validate_model import validate_model

(
    hard_position_accuracy,
    legal_piece_moves_accuracy,
    average_correct_plies,
    error_frequencies,
    hard_position_results,
    legal_piece_moves_results,
    sequence_results,
) = validate_model(
    model,
    max_batch_size=100,
    number_of_plies_to_generate=125,
    number_of_sequences=10,
    do_sequence_validation=True,
    tokens_per_ply=4,
    notation="xLANplus",
    left_padding=True,
)

In [ ]:
print(
    f"hard_position_accuracy: {hard_position_accuracy}\nLegal piece moves accuracy: {legal_piece_moves_accuracy}\nAverage correct plies: {average_correct_plies}\nError frequencies: {error_frequencies}\n"
)
print(
    f"hard_position_results: {hard_position_results}\nLegal piece moves results: {legal_piece_moves_results}\nSequence results: {sequence_results}"
)

In [ ]:
import json

file_name = modelname.split("/")[-1] + "_results.json"


results = {
    "hard_position_accuracy": hard_position_accuracy,
    "legal_piece_moves_accuracy": legal_piece_moves_accuracy,
    "average_correct_plies": average_correct_plies,
    "error_frequencies": error_frequencies,
    "hard_position_results": hard_position_results,
    "legal_piece_moves_results": legal_piece_moves_results,
    "sequence_results": sequence_results,
}


with open(f"./results/model_evaluations/{file_name}", "w") as f:
    json.dump(results, f)

# Correct and Incorrect Positions

## Correct Hard Positions and Legal Piece Move Positions for 1 Model

In [ ]:
import json

file_name = "Leon-Chess-71k-Plus_results.json"


with open(f"./results/model_evaluations/{file_name}", "r") as file:
    data = json.load(file)


legal_piece_moves_results = data["legal_piece_moves_results"]
hard_position_results = data["hard_position_results"]


correct_hard_positions = []
incorrect_hard_positions = []
correct_legal_piece_moves = []
incorrect_legal_piece_moves = []


for result in hard_position_results:

    position_id = result[0]
    correctness = result[2]
    if correctness:
        correct_hard_positions.append(position_id)
    else:
        incorrect_hard_positions.append(position_id)

for result in legal_piece_moves_results:
    position_id = result[0]
    correctness = result[3]
    if correctness:
        correct_legal_piece_moves.append(position_id)
    else:
        incorrect_legal_piece_moves.append(position_id)


result_summary = {
    "correct_hard_positions": correct_hard_positions,
    "incorrect_hard_positions": incorrect_hard_positions,
    "correct_legal_piece_moves": correct_legal_piece_moves,
    "incorrect_legal_piece_moves": incorrect_legal_piece_moves,
}


print("Correct Positions:", result_summary["correct_hard_positions"])
print("Incorrect Positions:", result_summary["incorrect_hard_positions"])
print("Correct Legal Piece Moves:", result_summary["correct_legal_piece_moves"])
print("Incorrect Legal Piece Moves:", result_summary["incorrect_legal_piece_moves"])

### Plot Chess Boards with resualts of the hard positions metric

In [ ]:
from src.validation.validate_position import get_position_by_id
from src.validation.validate_position import show_position_by_id
import json

file_name = "Leon-Chess-71k-Plus_results.json"


with open(f"./results/model_evaluations/{file_name}", "r") as file:
    data = json.load(file)

hard_position_results = data["hard_position_results"]

for result in hard_position_results:

    position_id = result[0]
    predicted_move = result[1]
    correctness = result[2]
    correctness_status = "Correct" if correctness else "Incorrect"

    print(f"Position ID: {position_id}")
    print(f"Predicted Move: {predicted_move}")
    print(f"Correctness: {correctness_status}")

    print(get_position_by_id(position_id, metric="hard_positions", notation="xLANplus"))
    show_position_by_id(position_id, metric="hard_positions", notation="xLANplus")
    print("\n" + "-" * 50 + "\n")

### Plot Chess Boards with resualts of the legal piece moves metric

In [ ]:
from src.validation.validate_position import get_position_by_id
from src.validation.validate_position import show_position_by_id

import json

file_name = "Leon-Chess-71k-Plus_results.json"


with open(f"./results/model_evaluations/{file_name}", "r") as file:
    data = json.load(file)

legal_piece_moves_results = data["legal_piece_moves_results"]

for result in legal_piece_moves_results:
    position_id = result[0]
    predicted_move = result[1]
    predicted_move_index = result[2]
    correctness = result[3]
    correctness_status = "Correct" if correctness else "Incorrect"

    print(f"Position ID: {position_id}")
    print(f"Predicted Move: {predicted_move}")
    print(f"Predicted Move Index: {predicted_move_index}")
    print(f"Correctness: {correctness_status}")

    print(get_position_by_id(position_id, metric="board_state", notation="xLANplus"))
    show_position_by_id(position_id, metric="board_state", notation="xLANplus")
    print("\n" + "-" * 50 + "\n")

## Correct Hard Positions and Legal Piece Move Positions for multiple Models

### Compares different Models on Hard Positions
- JSON Files needed

In [ ]:
import json
from src.validation.validate_position import get_position_by_id
from src.validation.validate_position import show_position_by_id


model_names = [
    "R1_GPT2_19k_4E_xLANplus",
    "R5_GPT2_71k_4E_xLANplus",
    "R2_GPT2_350k_4E_xLANplus",
    "R3_Mamba_19k_4E_xLANplus",
    "R6_Mamba_71k_4E_xLANplus",
    "R4_Mamba_350k_4E_xLANplus",
]
all_results = {}

for modelname in model_names:

    with open(f"./results/model_evaluations/{modelname}.json", "r") as file:
        data = json.load(file)

    hard_position_results = data["hard_position_results"]

    for result in hard_position_results:

        position_id = result[0]
        predicted_move = result[1]
        correctness = result[2]
        correctness_status = "Correct" if correctness else "Incorrect"
        if position_id not in all_results:
            all_results[position_id] = []
        all_results[position_id].append(
            {
                "model": modelname,
                "predicted_move": predicted_move,
                "correctness_status": correctness_status,
            }
        )


for position_id, results in all_results.items():
    print(f"Position ID: {position_id}")

    print(get_position_by_id(position_id, metric="hard_positions", notation="xLANplus"))
    show_position_by_id(position_id, metric="hard_positions", notation="xLANplus")
    for result in results:
        print(f"Model: {result['model']}")
        print(f"Predicted Move: {result['predicted_move']}")
        print(f"Correctness: {result['correctness_status']}")
        print("-" * 50)

### Compares different Models on Hard Positions
- JSON Files needed

In [ ]:
import json
from src.validation.validate_position import get_position_by_id
from src.validation.validate_position import show_position_by_id

model_names = [
    "R1_GPT2_19k_4E_xLANplus",
    "R5_GPT2_71k_4E_xLANplus",
    "R2_GPT2_350k_4E_xLANplus",
    "R3_Mamba_19k_4E_xLANplus",
    "R6_Mamba_71k_4E_xLANplus",
    "R4_Mamba_350k_4E_xLANplus",
]

all_results = {}

for modelname in model_names:
    with open(f"./results/model_evaluations/{modelname}.json", "r") as file:
        data = json.load(file)

    legal_piece_moves_results = data["legal_piece_moves_results"]
    for result in legal_piece_moves_results:
        position_id = result[0]
        predicted_moves = result[1]
        correct_moves = result[2]
        correctness = result[3]
        correctness_status = "Correct" if correctness else "Incorrect"

        if position_id not in all_results:
            all_results[position_id] = []

        all_results[position_id].append(
            {
                "model": modelname,
                "predicted_moves": predicted_moves,
                "correct_moves": correct_moves,
                "correctness_status": correctness_status,
            }
        )
for position_id, results in all_results.items():
    print(f"Position ID: {position_id}")
    print(get_position_by_id(position_id, metric="board_state", notation="xLANplus"))
    show_position_by_id(position_id, metric="board_state", notation="xLANplus")

    for result in results:
        print(f"Model: {result['model']}")
        print(f"Predicted Moves: {result['predicted_moves']}")
        print(f"Correct Moves: {result['correct_moves']}")
        print(f"Correctness: {result['correctness_status']}")
        print("-" * 50)

# Plot the Evaluations saved as CSV

## Legal Piece Moves for multiple Models

In [ ]:
import csv
from src.validation.validate_position import get_position_by_id
from src.validation.validate_position import show_position_by_id


model_names = [
    "V27_GPT2_19k_4E_xLAN_legal_piece_moves_results.csv",
    "V28_GPT2_71k_4E_xLAN_legal_piece_moves_results.csv",
    "V32_GPT2_350k_4E_xLAN_legal_piece_moves_results.csv",
    "V29_GPT2_19k_4E_xLANplus_legal_piece_moves_results.csv",
    "V30_GPT2_71k_4E_xLANplus_legal_piece_moves_results.csv",
    "V31_GPT2_350k_4E_xLANplus_legal_piece_moves_results.csv",
]
all_results = {}


for modelname in model_names:

    with open(f"./results/model_evaluations/{modelname}", "r") as file:

        reader = csv.DictReader(file)

        for row in reader:
            position_id = row["Position ID"]
            predicted_moves = row["Predicted Move"]
            correct_moves = row["Correct Moves"]
            correctness = row["Correctness"]
            correctness_status = (
                "Correct" if correctness.lower() == "true" else "Incorrect"
            )
            if position_id not in all_results:
                all_results[position_id] = []
            all_results[position_id].append(
                {
                    "model": modelname,
                    "predicted_moves": predicted_moves,
                    "correct_moves": correct_moves,
                    "correctness_status": correctness_status,
                }
            )


for position_id, results in all_results.items():
    position_id = position_id.strip()
    position_id = int(position_id)
    print(f"Position ID: {position_id}")

    print(get_position_by_id(position_id, metric="board_state", notation="xLANplus"))
    show_position_by_id(position_id, metric="board_state", notation="xLANplus")
    for result in results:

        print(f"Model: {result['model']}")
        print(f"Predicted Moves: {result['predicted_moves']}")
        print(f"Correct Moves: {result['correct_moves']}")
        print(f"Correctness: {result['correctness_status']}")
        print("-" * 50)

## Plot the accuracies as Bar Plot

In [ ]:
import csv

model_names = [
    "V27_GPT2_19k_4E_xLAN_legal_piece_moves_results.csv",
    "V29_GPT2_19k_4E_xLANplus_legal_piece_moves_results.csv",
    "V28_GPT2_71k_4E_xLAN_legal_piece_moves_results.csv",
    "V30_GPT2_71k_4E_xLANplus_legal_piece_moves_results.csv",
    "V32_GPT2_350k_4E_xLAN_legal_piece_moves_results.csv",
    "V31_GPT2_350k_4E_xLANplus_legal_piece_moves_results.csv",
]

accuracy_results = {}

for modelname in model_names:
    total_positions = 0
    correct_predictions = 0

    with open(f"./results/model_evaluations/{modelname}", "r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            total_positions += 1
            correctness = row["Correctness"]
            if correctness.lower() == "true":
                correct_predictions += 1

    accuracy = correct_predictions / total_positions
    accuracy_results[modelname] = accuracy

In [ ]:
plot_bars_csv(
    accuracy_results,
    "Legal Piece Moves Accuracy Comparison xLAN vs xLAN+",
    "Legal Piece Moves Accuracy (%)",
)

## Hard Positions for multiple Models

In [ ]:
import csv
from src.validation.validate_position import get_position_by_id
from src.validation.validate_position import show_position_by_id



# Dictionary to store results for each model by position ID


model_names = [
    "V27_GPT2_19k_4E_xLAN_hard_positions_results.csv",
    "V28_GPT2_71k_4E_xLAN_hard_positions_results.csv",
    "V32_GPT2_350k_4E_xLAN_hard_positions_results.csv",
    "V29_GPT2_19k_4E_xLANplus_hard_positions_results.csv",
    "V30_GPT2_71k_4E_xLANplus_hard_positions_results.csv",
    "V31_GPT2_350k_4E_xLANplus_hard_positions_results.csv",
]


all_results = {}



# Iterate through each model


for modelname in model_names:


    # Load the CSV data for the current model

    with open(f"./results/model_evaluations/{modelname}", "r") as file:

        reader = csv.DictReader(file)

        for row in reader:

            position_id = row["Position ID"]

            predicted_move = row["Predicted Move"]

            correctness = row["Correctness"]

            correctness_status = (
                "Correct" if correctness.lower() == "true" else "Incorrect"
            )

            if position_id not in all_results:

                all_results[position_id] = []

            all_results[position_id].append(
                {
                    "model": modelname,
                    "predicted_move": predicted_move,
                    "correctness_status": correctness_status,
                }
            )



# Print the results for each position


for position_id, results in all_results.items():

    position_id = position_id.strip()

    # Convert string to number

    position_id = int(position_id)

    print(f"Position ID: {position_id}")


    print(get_position_by_id(position_id, metric="hard_positions", notation="xLANplus"))

    show_position_by_id(position_id, metric="hard_positions", notation="xLANplus")


    for result in results:


        print(f"Model: {result['model']}")

        print(f"Predicted Move: {result['predicted_move']}")

        print(f"Correctness: {result['correctness_status']}")

        print("-" * 50)

## Plot the accuracies as Bar Plot

In [ ]:
import csv

# Dictionary to store results for each model by position ID
model_names = [
    "V27_GPT2_19k_4E_xLAN_hard_positions_results.csv",
    "V29_GPT2_19k_4E_xLANplus_hard_positions_results.csv",
    "V28_GPT2_71k_4E_xLAN_hard_positions_results.csv",
    "V30_GPT2_71k_4E_xLANplus_hard_positions_results.csv",
    "V32_GPT2_350k_4E_xLAN_hard_positions_results.csv",
    "V31_GPT2_350k_4E_xLANplus_hard_positions_results.csv",
]
accuracy_results = {}

for modelname in model_names:
    total_positions = 0
    correct_predictions = 0
    # Load the CSV data for the current model
    with open(f"./results/model_evaluations/{modelname}", "r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            total_positions += 1
            correctness = row["Correctness"]
            if correctness.lower() == "true":
                correct_predictions += 1
    accuracy = correct_predictions / total_positions
    accuracy_results[modelname] = accuracy

In [ ]:
plot_bars_csv(
    accuracy_results,
    "Hard Position Accuracy Comparison xLAN vs xLAN+",
    "Hard Position Accuracy (%)",
)

# Bar Plots for IDs in Legal Piece

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np

model_names = [
    # "R1_GPT2_19k_4E_xLANplus",
    # "R5_GPT2_71k_4E_xLANplus",
    "R2_GPT2_350k_4E_xLANplus",
    # "R3_Mamba_19k_4E_xLANplus",
    # "R6_Mamba_71k_4E_xLANplus",
    "R4_Mamba_350k_4E_xLANplus",
]

all_results = {}

# Iterate through each model
for modelname in model_names:
    # Load the JSON data for the current model
    with open(f"./results/model_evaluations/{modelname}.json", "r") as file:
        data = json.load(file)
    # Extract the required lists
    legal_piece_moves_results = data["legal_piece_moves_results"]
    # Process the data to get the positions and print the required details

    for result in legal_piece_moves_results:
        position_id = result[0]
        predicted_moves = result[1]
        correct_moves = result[2]
        correctness = result[3]
        correctness_status = "Correct" if correctness else "Incorrect"
        if position_id not in all_results:
            all_results[position_id] = []
        all_results[position_id].append(
            {
                "model": modelname,
                "correctness_status": correctness_status,
            }
        )

# Separate the results into odd and even position IDs
odd_results = {k: v for k, v in all_results.items() if int(k) % 2 != 0}
even_results = {k: v for k, v in all_results.items() if int(k) % 2 == 0}


# Function to prepare data for plotting
def prepare_plot_data(results, model_name):
    position_ids = []
    statuses = []

    for position_id, result_list in results.items():
        for result in result_list:
            if result["model"] == model_name:
                position_ids.append(position_id)
                statuses.append(result["correctness_status"])

    return position_ids, statuses


# Function to create a bar plot for a specific model
def create_bar_plot(model_name, position_ids, statuses, title):
    unique_positions = list(set(position_ids))
    bar_width = 0.35
    index = np.arange(len(unique_positions))

    correct_counts = [0] * len(unique_positions)
    incorrect_counts = [0] * len(unique_positions)

    for i, pos_id in enumerate(unique_positions):
        count_correct = sum(
            1
            for j in range(len(position_ids))
            if position_ids[j] == pos_id and statuses[j] == "Correct"
        )
        count_incorrect = sum(
            1
            for j in range(len(position_ids))
            if position_ids[j] == pos_id and statuses[j] == "Incorrect"
        )
        correct_counts[i] = count_correct
        incorrect_counts[i] = count_incorrect

    fig, ax = plt.subplots(figsize=(14, 8))

    ax.bar(index, correct_counts, bar_width, label="Correct")
    ax.bar(index, incorrect_counts, bar_width, bottom=correct_counts, label="Incorrect")

    ax.set_xlabel("Position IDs")
    ax.set_ylabel("Count")
    ax.set_title(f"{title} - {model_name}")
    ax.set_xticks(index)
    ax.set_xticklabels(unique_positions, rotation=90)
    ax.legend()

    plt.tight_layout()
    plt.show()


# Create bar plots for each model for odd and even results
for model_name in model_names:
    odd_position_ids, odd_statuses = prepare_plot_data(odd_results, model_name)
    even_position_ids, even_statuses = prepare_plot_data(even_results, model_name)
    create_bar_plot(
        model_name,
        odd_position_ids,
        odd_statuses,
        "Correctness Status for Odd Position IDs",
    )
    create_bar_plot(
        model_name,
        even_position_ids,
        even_statuses,
        "Correctness Status for Even Position IDs",
    )

# Bar Plots for all 3

In [ ]:
import json

model_names = [
    "R1_GPT2_19k_4E_xLANplus",
    "R3_Mamba_19k_4E_xLANplus",
    "R5_GPT2_71k_4E_xLANplus",
    "R6_Mamba_71k_4E_xLANplus",
    "R2_GPT2_350k_4E_xLANplus",
    "R4_Mamba_350k_4E_xLANplus",
]

all_hard_position_accuracies = {}
all_legal_piece_moves_accuracies = {}
all_average_correct_plies = {}

for modelname in model_names:
    with open(f"./results/model_evaluations/{modelname}.json", "r") as file:
        data = json.load(file)

    hard_position_accuracy = data["hard_position_accuracy"]
    legal_piece_moves_accuracy = data["legal_piece_moves_accuracy"]
    average_correct_plies = data["average_correct_plies"]
    all_hard_position_accuracies[modelname] = hard_position_accuracy
    all_legal_piece_moves_accuracies[modelname] = legal_piece_moves_accuracy
    all_average_correct_plies[modelname] = average_correct_plies

print(all_hard_position_accuracies)
print(all_legal_piece_moves_accuracies)
print(all_average_correct_plies)

In [ ]:
# Plot the results
plot_bars(
    all_hard_position_accuracies,
    "Hard Position Accuracy Comparison GPT-2 vs Mamba",
    "Hard Position Accuracy (%)",
)

In [ ]:
# Plot the results
plot_bars(
    all_legal_piece_moves_accuracies,
    "Legal Piece Moves Accuracy Comparison GPT-2 vs Mamba",
    "Legal Piece Moves Accuracy (%)",
)

# Plot the results
plot_bars(
    all_average_correct_plies,
    "Average Number of Correct Plies Comparison GPT-2 vs Mamba",
    "Average Number of Correct Plies",
)

# Plot Averge Number of correct plies

In [ ]:
# Raw data:
xLAN_19k = 2.31
xLAN_71k = 14.42
xLAN_350k = 35.98
xLANplus_19k = 10.25
xLANplus_71k = 24.15
xLANplus_350k = 44.80

# Plot the results
plot_bars_avg(
    {
        "xLAN_19k": xLAN_19k,
        "xLANplus_19k": xLANplus_19k,
        "xLAN_71k": xLAN_71k,
        "xLANplus_71k": xLANplus_71k,
        "xLAN_350k": xLAN_350k,
        "xLANplus_350k": xLANplus_350k,
    },
    "Average Number of Correct Plies Comparison xLAN vs xLAN+",
    "Average Number of Correct Plies",
)

# Plot  different Plus notations

In [ ]:
# Raw data average number of correct plies for xLAN+ variations:
xLAN_71k_max = 17.49
xLANchk_71k_max = 23.56
xLANcap_71k_max = 25.00
xLANplus_71k_max = 25.86

# Plot the results

plot_bars_avg(
    {
        "xLAN_71k": xLAN_71k_max,
        "xLANchk_71k": xLANchk_71k_max,
        "xLANcap_71k": xLANcap_71k_max,
        "xLANplus_71k": xLANplus_71k_max,
    },
    "Average Number of Correct Plies Comparison xLAN+ Variations",
    "Average Number of Correct Plies",
)

In [ ]:
# raw data hard position accuracy for xLAN+ variations:
xLAN_71k = 0.78
xLANchk_71k = 0.79
xLANcap_71k = 0.81
xLANplus_71k = 0.81

# Plot the results
plot_bars_avg(
    {
        "xLAN_71k": xLAN_71k,
        "xLANchk_71k": xLANchk_71k,
        "xLANcap_71k": xLANcap_71k,
        "xLANplus_71k": xLANplus_71k,
    },
    "Hard Position Accuracy Comparison xLAN+ Variations",
    "Hard Position Accuracy (%)",
)

In [ ]:
# raw data legal piece moves accuracy for xLAN+ variations:
xLAN_71k = 0.79
xLANchk_71k = 0.78
xLANcap_71k = 0.80
xLANplus_71k = 0.81

# Plot the results

plot_bars_avg(
    {
        "xLAN_71k": xLAN_71k,
        "xLANchk_71k": xLANchk_71k,
        "xLANcap_71k": xLANcap_71k,
        "xLANplus_71k": xLANplus_71k,
    },
    "Legal Piece Moves Accuracy Comparison xLAN+ Variations",
    "Legal Piece Moves Accuracy (%)",
)